In [ ]:
import json
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from config import snowflake_conn_prop
import pandas as pd

session = Session.builder.configs(snowflake_conn_prop).create()

In [ ]:
# how I would do in pandas
df = pd.read_csv('./medium_data.csv', index_col=0)
df.head(5)
# 6508 urls

In [ ]:
session.file.put('./medium_data.csv', '@~/', overwrite=True)
schema = StructType([
  StructField("id", IntegerType()),
  StructField("url", StringType()),
  StructField("title", StringType()),
  StructField("subtitle", StringType()),
  StructField("image", StringType()),
  StructField("claps", IntegerType()),
  StructField("responses", VariantType()),
  StructField("reading_time", IntegerType()),
  StructField("publication", StringType()),
  StructField("date", DateType())
])
df = session.read.option("skip_header", 1).option("field_delimiter", ",").option("FIELD_OPTIONALLY_ENCLOSED_BY", "\"").schema(schema).csv('@~/medium_data.csv')
df_pd = df.to_pandas().set_index('ID')

In [ ]:
df_pd = df_pd.loc[~df_pd["URL"].str.startswith('https://')]

In [ ]:
df_tldr = df.select(
    df.url, df.title, call_builtin('url_resolve_and_summarize', df.url).alias("TL_DR")
    )

df_tldr.write.save_as_table('medium_tldr', mode="overwrite")